In [1]:
import pickle
import numpy as np

def load_cifar10_batch(filename):
    with open(filename, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')

    X = batch[b'data'].astype(np.float32)     # shape (10000, 3072)
    y = np.array(batch[b'labels'], dtype=np.int64)

    X /= 255.0  # normalize

    return X, y


def load_cifar10():
    X_list = []
    y_list = []

    for i in range(1, 6):
        X, y = load_cifar10_batch(f"data_batch_{i}")
        X_list.append(X)
        y_list.append(y)

    X_train = np.concatenate(X_list)
    y_train = np.concatenate(y_list)

    X_test, y_test = load_cifar10_batch("test_batch")

    return X_train, y_train, X_test, y_test

In [2]:
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def softmax(x):
    x = x - np.max(x, axis=1, keepdims=True)
    expx = np.exp(x)
    return expx / np.sum(expx, axis=1, keepdims=True)

def cross_entropy(pred, labels):
    n = labels.shape[0]
    return -np.sum(np.log(pred[np.arange(n), labels])) / n

In [3]:
class ScratchNN:
    def __init__(self, input_dim=3072, h1=512, h2=256, out=10, lr=0.001):
        self.lr = lr

        self.W1 = np.random.randn(input_dim, h1) / np.sqrt(input_dim)
        self.b1 = np.zeros((1, h1))

        self.W2 = np.random.randn(h1, h2) / np.sqrt(h1)
        self.b2 = np.zeros((1, h2))

        self.W3 = np.random.randn(h2, out) / np.sqrt(h2)
        self.b3 = np.zeros((1, out))

    def forward(self, x):
        self.x = x
        self.z1 = x @ self.W1 + self.b1
        self.a1 = relu(self.z1)

        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = relu(self.z2)

        self.z3 = self.a2 @ self.W3 + self.b3
        self.out = softmax(self.z3)

        return self.out

    def backward(self, labels):
        n = labels.shape[0]

        dZ3 = self.out
        dZ3[np.arange(n), labels] -= 1
        dZ3 /= n

        dW3 = self.a2.T @ dZ3
        db3 = np.sum(dZ3, axis=0, keepdims=True)

        dA2 = dZ3 @ self.W3.T
        dZ2 = dA2 * relu_deriv(self.z2)
        dW2 = self.a1.T @ dZ2
        db2 = np.sum(dZ2, axis=0, keepdims=True)

        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * relu_deriv(self.z1)
        dW1 = self.x.T @ dZ1
        db1 = np.sum(dZ1, axis=0, keepdims=True)

        # update
        self.W3 -= self.lr * dW3
        self.b3 -= self.lr * db3

        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2

        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

In [4]:
def train(model, X, y, epochs=10, batch_size=128):
    N = X.shape[0]

    for epoch in range(epochs):
        idx = np.random.permutation(N)
        X = X[idx]
        y = y[idx]

        losses = []

        for i in range(0, N, batch_size):
            xb = X[i:i+batch_size]
            yb = y[i:i+batch_size]

            pred = model.forward(xb)
            loss = cross_entropy(pred, yb)
            model.backward(yb)

            losses.append(loss)

        print(f"Epoch {epoch+1} | Loss = {np.mean(losses):.4f}")

In [5]:
def accuracy(model, X, y):
    preds = np.argmax(model.forward(X), axis=1)
    return np.mean(preds == y) * 100

In [6]:
X_train, y_train, X_test, y_test = load_cifar10()

model = ScratchNN(lr=0.001)

train(model, X_train, y_train, epochs=10, batch_size=128)

print("Training Accuracy:", accuracy(model, X_train, y_train))
print("Test Accuracy:", accuracy(model, X_test, y_test))

Epoch 1 | Loss = 2.2440
Epoch 2 | Loss = 2.1557
Epoch 3 | Loss = 2.0922
Epoch 4 | Loss = 2.0438
Epoch 5 | Loss = 2.0058
Epoch 6 | Loss = 1.9758
Epoch 7 | Loss = 1.9510
Epoch 8 | Loss = 1.9302
Epoch 9 | Loss = 1.9126
Epoch 10 | Loss = 1.8971
Training Accuracy: 34.582
Test Accuracy: 34.18
